In [1]:
from PIL import Image
from create_dataset import create_coco, create_object2color

import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm
import datetime
import json
%matplotlib inline

label_name = os.listdir('./Dataset/sample_train_label/')
label_path = ['./Dataset/sample_train_label/'+name for name in label_name]
label_path.sort()
############# To Delete ###############
label_path = label_path
############# To Delete ###############

img_name = os.listdir('./Dataset/sample_train_color/')
img_path = ['./Dataset/sample_train_color/'+name for name in img_name]
img_path.sort()
############# To Delete #1#############
img_path = img_path
############# To Delete ###############

In [2]:
object2color = np.load('./Dataset/object2color.npy')
create_coco(img_path, label_path, './Dataset/data.json', object2color=object2color, crop_size=[75, 75, 28, 28], down_scale=4)
#create_coco(img_path, label_path, './Dataset/data.json')

Creating image list...
object2color is obtained...


UnboundLocalError: local variable 'class_instance' referenced before assignment